# Job 1

In [ ]:
import org.apache.spark

### Schema definitions

In [ ]:
import org.apache.spark.sql.types._
import java.sql.Timestamp

val reviewSchema = StructType(
  Seq(
    StructField("user_id",  StringType,            nullable = true),
    StructField("name",     StringType,            nullable = true),
    StructField("time",     LongType,              nullable = false),
    StructField("rating",   DoubleType,            nullable = true),
    StructField("text",     StringType,            nullable = true),
    StructField("pics",     ArrayType(StringType), nullable = true),
    StructField("resp",     StructType(
      Seq(
        StructField("time", LongType,              nullable = false),
        StructField("text", StringType,            nullable = true)
      )
    ),                                             nullable = true),
    StructField("gmap_id",  StringType,            nullable = false),
  )
)

case class Response(time: Timestamp, text: Option[String])

case class Review(
  user_id: Option[String],
  name: Option[String],
  time: Timestamp,
  rating: Option[Double],
  text: Option[String],
  pics: Seq[String],
  resp: Option[Response],
  gmap_id: String
)

In [ ]:
val metadataSchema = StructType(
  Seq(
    StructField("name",             StringType,                                 nullable = true),
    StructField("address",          StringType,                                 nullable = true),
    StructField("gmap_id",          StringType,                                 nullable = false),
    StructField("description",      StringType,                                 nullable = true),
    StructField("latitude",         DoubleType,                                 nullable = false),
    StructField("longitude",        DoubleType,                                 nullable = false),
    StructField("category",         ArrayType(StringType),                      nullable = true),
    StructField("avg_rating",       DoubleType,                                 nullable = false),
    StructField("num_of_reviews",   IntegerType,                                nullable = false),
    StructField("price",            StringType,                                 nullable = false),
    StructField("hours",            ArrayType(ArrayType(StringType)),           nullable = true),
    StructField("MISC",             MapType(StringType, ArrayType(StringType)), nullable = false),
    StructField("state",            StringType,                                 nullable = true),
    StructField("relative_results", ArrayType(StringType),                      nullable = true),
    StructField("url",              StringType,                                 nullable = false),
  )
)

case class Metadata(
  name: Option[String],
  address: Option[String],
  gmap_id: String,
  description: Option[String],
  latitude: Double,
  longitude: Double,
  category: Seq[String],
  avg_rating: Double,
  num_of_reviews: Int,
  price: String,
  hours: Seq[Seq[String]],
  MISC: Map[String, Seq[String]],
  state: Option[String],
  relative_results: Seq[String],
  url: String
)

### Dataset load and parse

In [ ]:
import java.nio.file.Paths
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions.from_unixtime

val projectDir: String = Paths.get(System.getProperty("user.dir")).getParent.getParent.getParent.toString
val reviewsPath = s"$projectDir/dataset/sample-reviews.ndjson"
val metadataPath = s"$projectDir/dataset/metadata.ndjson"

val spark = SparkSession.builder()
  .appName("NDJSON Reader")
  .master("local[*]") // Needed in local mode
  .getOrCreate()

val reviewsDf = spark.read
  .schema(reviewSchema)
  .json(reviewsPath)
  .withColumn("pics", when (col("pics") isNull, array()) otherwise col("pics"))
  .withColumn("time", from_unixtime(col("time") / 1000).cast("timestamp"))
  .withColumn("resp", 
    when (
      col("resp") isNotNull, 
      struct(
        from_unixtime(col("resp.time") / 1000).cast("timestamp").alias("time"),
        col("resp.text").cast(StringType).alias("text")
      )
    ) otherwise lit(null)
  )
  .as[Review]

val metadataDf = spark.read
  .schema(metadataSchema)
  .json(metadataPath)
  .withColumn("category", when (col("category") isNull, array()) otherwise col("category"))
  .withColumn("hours", when (col("hours") isNull, array()) otherwise col("hours"))
  .withColumn("relative_results", when (col("relative_results") isNull, array()) otherwise col("relative_results"))
  .as[Metadata]

reviewsDf.printSchema()
metadataDf.printSchema()

// Unforturnately, it seems that Spark does not support case classes in RDDs. It throws ArrayStoreException
// when trying to collect the RDD... [see also [here](https://github.com/adtech-labs/spylon-kernel/issues/40)]
val reviewsRdd = reviewsDf.rdd
  .map(Review.unapply(_).get)
  .map { case review @ (_, _, _, _, _, _, resp, _) => review.copy(_7 = resp.map(Response.unapply(_).get)) }
val metaRdd = metadataDf.rdd.map(Metadata.unapply).map(_.get)

L'obiettivo di questo job è capire, anno per anno, se una maggiore frequenza nella risposta alle recensioni ha un impatto sulla valutazione media ricevuta.
In particolare:
- Per ogni anno e business si calcola la media delle recensioni, il rate e il tempo medio di risposta;
- Sulla base del rate e del tempo medio di risposta viene calcolata un attributo aggiuntivo “response strategy” che categorizza il business in un particolare anno in 4 categorie (“Rapid and frequent”, “Slow but frequent”, “Occasional” o “Rare or none”);
- Aggregazione in base alla "response strategy", l'anno e lo stato per ottenere il rate medio e il numero di business all'interno della categoria.

---

The goal of this job is to understand, year by year, whether greater frequency in responding to reviews has an impact on the average rating received.

Specifically:

- For each year and business, the average rating, rate, and average response time are calculated;
- Based on the rate and average response time, an additional attribute "response strategy" is calculated that categorizes the business in a particular year into four categories ("Rapid and frequent," "Slow but frequent," "Occasional," or "Rare or none");
- Aggregation based on the "response strategy," year, and state to get the average rate and number of businesses within the category.

---

**Metadata**: (name, address, <ins>gmap_id</ins>, description, latitude, longitude, category, avg_rating, num_of_reviews, price, hours, misc, state, relative_results, url)

**Review**: (user_id, name, time, rating, text, pics, responses, <ins>gmap_id</ins>)

---

In [ ]:
import java.util.concurrent.TimeUnit
import org.apache.spark.sql.SaveMode

In [ ]:
def responseStrategy(avgResponseRate: Double, avgResponseTime: Double): String =
  (avgResponseRate, avgResponseTime) match {
    case (rr, rt) if rr >= 0.5 && rt <= 4 * 24 => "Rapid and frequent"
    case (rr, rt) if rr >= 0.5 => "Slow but frequent"
    case (rr, _)  if rr >= 0.15 => "Occasional"
    case _ => "Rare or none"
  }

In [ ]:

def toState(address: Option[String]): String = 
  address.flatMap { addr =>
    // This regex captures the state abbreviation between a comma and the ZIP code
    val StateRegex = """,\s*([A-Z]{2})\s+\d{5}""".r
    StateRegex.findFirstMatchIn(addr).map(_.group(1))
  }.getOrElse("Unknown")

In [ ]:
val businessesStates = metaRdd
  .filter(_._2.isDefined) // leave only businesses with a defined address
  .map(b => b._3 -> toState(b._2))

In [ ]:
val reviewsInfo = reviewsRdd
  .filter(_._4.isDefined) // filter out reviews without a rating
  .map { case (_, _, time, rating, _, _, resp, id) => id -> (time, rating.get, resp) }
  .join(businessesStates) // [(gmap_id, ((timestamp, rating, response), state))*]
  .map { case (id, ((time, rating, resp), state)) => (time.toLocalDateTime.getYear, id, state) -> (time, rating, resp) }
  .aggregateByKey((0.0, 0, 0L, 0))( // (ratings sum, #responses, sum of response time differences, #reviews)
    (acc, v) => {
      val (sumRatings, numResponses, sumResponseTimes, totalReviews) = acc
      val (time, rating, response) = v
      (
        sumRatings + rating,
        numResponses + (if (response.isDefined) 1 else 0),
        sumResponseTimes + (if (response.isDefined) response.get._1.getTime - time.getTime else 0L),
        totalReviews + 1
      )
    },
    (r1, r2) => (r1._1 + r2._1, r1._2 + r2._2, r1._3 + r2._3, r1._4 + r2._4)
  )
  .mapValues { case (sumRatings, numResponses, sumResponseTimes, totalReviews) => 
    (
      sumRatings / totalReviews,
      numResponses.toDouble / totalReviews,
      if (numResponses > 0) TimeUnit.MILLISECONDS.toHours(sumResponseTimes / numResponses) else Double.PositiveInfinity
    )
  } // [((year, gmap_id, state), (avg_rating, response_rate, avg_response_time))*]
  .mapValues { case (avgRating, responseRate, avgResponseTime) => 
    (avgRating, responseRate, avgResponseTime, responseStrategy(responseRate, avgResponseTime)) 
  } // [((year, gmap_id, state), (avg_rating, response_rate, avg_response_time, response_strategy))*]

In [ ]:
val outcome = reviewsInfo
  .map { case ((year, _, state), (avgRating, _, _, responseStrategy)) => (year, state, responseStrategy) -> avgRating }
  .aggregateByKey((0.0, 0))((acc, v) => (acc._1 + v, acc._2 + 1), (r1, r2) => (r1._1 + r2._1, r1._2 + r2._2))
  .mapValues { case (sumRatings, totalBusinesses) => sumRatings / totalBusinesses }

In [ ]:
val outputDirPath = s"$projectDir/output"
val outputPath = s"$outputDirPath/job1-output"

In [ ]:
outcome.map { case ((year, state, responseStrategy), avgRating) => (year, state, responseStrategy, avgRating) }
  .coalesce(1)
  .toDF("year", "state", "response_strategy", "avg_rating")
  .write.format("csv").option("header", "true")
  .mode(SaveMode.Overwrite)
  .save(s"file://$outputPath")

### Results

In [ ]:
%%python
import glob
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

files = glob.glob("/Users/lucatassi/Projects/big-data/big-data-project/output/job1-output/*.csv")

if not files:
    raise FileNotFoundError("No CSV files found")

df = pd.read_csv(files[0])

# Set seaborn style
sns.set(style="whitegrid")

# --- 1. Faceted bar charts by state ---
g = sns.catplot(
    data=df,
    x="response_strategy",
    y="avg_rating",
    hue="year",
    col="state",
    kind="bar",
    col_wrap=4,
    height=4,
    sharey=False,
    palette="muted"
)
g.set_xticklabels(rotation=30)
g.fig.suptitle("Average Rating by Response Strategy, Year & State", y=1.03)
plt.show()

In [ ]:
%%python

# --- 2. Heatmap: average rating by state and response_strategy ---
pivot_df = df.pivot_table(
    index="state",
    columns="response_strategy",
    values="avg_rating",
    aggfunc='mean'
)

plt.figure(figsize=(12,8))
sns.heatmap(
    pivot_df,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    center=df["avg_rating"].mean()
)
plt.title("Heatmap: Average Rating by State and Response Strategy")
plt.ylabel("State")
plt.xlabel("Response Strategy")
plt.xticks(rotation=30)
plt.show()

In [ ]:
%%python

# --- 3. Boxplot: distribution of avg_rating by state and response_strategy ---
plt.figure(figsize=(16,6))
sns.boxplot(
    data=df,
    x="state",
    y="avg_rating",
    hue="response_strategy",
    palette="Set2"
)
plt.xticks(rotation=90)
plt.title("Boxplot of Average Ratings by State and Response Strategy")
plt.legend(title="Response Strategy", bbox_to_anchor=(1.05, 1), loc=2)
plt.tight_layout()
plt.show()

In [ ]:
%%python
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# --- 1. Choropleth ---
fig_choro = px.choropleth(
    df,
    locations="state",
    locationmode="USA-states",
    color="avg_rating",
    scope="usa",
    animation_frame="year",
    title="Choropleth: Avg Rating by State"
)
fig_choro.show()

# --- 2. Bar Chart ---
fig_bar = px.bar(
    df.groupby("state")["avg_rating"].mean().reset_index(),
    x="state",
    y="avg_rating",
    title="Average Rating by State"
)
fig_bar.show()

# --- 3. Line Chart ---
fig_line = px.line(
    df.groupby(["year", "state"])["avg_rating"].mean().reset_index(),
    x="year",
    y="avg_rating",
    color="state",
    title="Rating Trends Over Time"
)
fig_line.show()

# --- 4. Scatter Plot ---
fig_scatter = px.scatter(
    df,
    x="avg_rating",
    y=np.random.uniform(1, 100, len(df)),
    color="state",
    title="Scatter Plot Example"
)
fig_scatter.show()

# --- 5. Heatmap ---
pivot = df.pivot_table(
    index="state", columns="year", values="avg_rating", aggfunc="mean"
)
fig_heatmap = go.Figure(
    data=go.Heatmap(
        z=pivot.values,
        x=pivot.columns.astype(str),
        y=pivot.index,
        colorscale="Viridis"
    )
)
fig_heatmap.update_layout(title="Heatmap: Avg Ratings by Year and State")
fig_heatmap.show()

# --- 6. Pie Chart ---
fig_pie = px.pie(
    df,
    names="state",
    title="State Distribution in Data"
)
fig_pie.show()

fig_choro.write_html("choropleth.html")

print("✅ All charts generated successfully.")